<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#Исследование-задачи" data-toc-modified-id="Исследование-задачи-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Исследование задачи</a></span></li><li><span><a href="#Борьба-с-дисбалансом" data-toc-modified-id="Борьба-с-дисбалансом-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Борьба с дисбалансом</a></span><ul class="toc-item"><li><span><a href="#В-качестве-эксперимента-попробуем-не-писать-самостоятельно-функцию,-а-воспользоваться-аргументом-class_weight-при-вызове-логистической-регрессии-и-случайного-леса.-В-аргумент-запишем,-что-у-отрицательных-объектов-будет-вес-1,-а-у-положительных-4" data-toc-modified-id="В-качестве-эксперимента-попробуем-не-писать-самостоятельно-функцию,-а-воспользоваться-аргументом-class_weight-при-вызове-логистической-регрессии-и-случайного-леса.-В-аргумент-запишем,-что-у-отрицательных-объектов-будет-вес-1,-а-у-положительных-4-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>В качестве эксперимента попробуем не писать самостоятельно функцию, а воспользоваться аргументом class_weight при вызове логистической регрессии и случайного леса. В аргумент запишем, что у отрицательных объектов будет вес 1, а у положительных 4</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Тестирование-модели" data-toc-modified-id="Тестирование-модели-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Тестирование модели</a></span></li><li><span><a href="#Чек-лист-готовности-проекта" data-toc-modified-id="Чек-лист-готовности-проекта-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист готовности проекта</a></span></li></ul></div>

# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

## Подготовка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler 
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.utils import shuffle
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer

In [2]:
try:

    data = pd.read_csv('/datasets/Churn.csv')
except:

    data = pd.read_csv('Churn.csv')
    
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


Проверим типы и заполненность данных

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


Видим, что есть пропуски в поле Tenure. Заменим их медианным значением для датасета

In [4]:
data.loc[data['Tenure'].isna(),'Tenure'] = data["Tenure"].mode()[0]

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB




Переведем имена колонок с camelcase на snakecase

In [6]:
def change_case(str):
#если элемент списка заглавная буква, меняем ее на строчную и добавляем нижнее подчеркивание перед ней
#по окончании цикла убираем нижнее подчеркивание в начале строки
    return ''.join(['_'+i.lower() if i.isupper()
               else i for i in str]).lstrip('_')

data.columns = data.columns.to_series().apply(change_case)


Теперь проверим есть ли в данных дубликаты

In [7]:
data.duplicated().sum()

0

Изучим данные в столбцах с категориальными значениями

Посмотрим есть ли в полях неявные дубликаты.

Фамилия клиента нас интересовать не будет, поэтому проверим поля geography и gender

In [8]:
print(data['gender'].unique())
print(data['geography'].unique())

['Female' 'Male']
['France' 'Spain' 'Germany']


In [9]:
data.describe()

,row_number,customer_id,credit_score,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,exited
count,10000.00000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,4.634300,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,2.989725,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,2.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,4.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


Минимальные и максимальные значения в полях age, tenure, balance, num_of_products и estimated_salary не вызывают вопросов

Подготовим данные к обучению. Поля row_number, customer_id и surname не помогут нам предсказать уход клиента

In [10]:
data.drop(['row_number', 'customer_id', 'surname'], axis = 1, inplace = True)

In [11]:
#КОД РЕВЬЮЕРА

data.isna().sum()

credit_score        0
geography           0
gender              0
age                 0
tenure              0
balance             0
num_of_products     0
has_cr_card         0
is_active_member    0
estimated_salary    0
exited              0
dtype: int64

In [12]:
#data_ohe = pd.get_dummies(data, columns = ['geography', 'gender'], drop_first=True)

Разделим датасет на features и target 

In [13]:
features_unbalanced = data.drop(['exited'], axis = 1)
target_unbalanced = data['exited']

## Исследование задачи

Проверим дисбаланс классов

In [14]:
target_unbalanced.value_counts(normalize = True)

0    0.7963
1    0.2037
Name: exited, dtype: float64

Видим, что отрицательных объектов гораздо больше, чем положительных. Но для начала попробуем обучить модель без учёта дисбаланса.

Разделим датасет на обучающую, валидационную и тестовую выборки, дважды применив функцию train_test_split. 

В train_test_split также применим аргумент stratify для одинакового распределения таргетов

In [15]:
features = features_unbalanced.copy()
target = target_unbalanced.copy()

features_train, features_valid_test, target_train, target_valid_test = train_test_split(
    features, target, test_size=0.4, random_state=12345, stratify = target)

features_valid, features_test, target_valid, target_test = train_test_split(
    features_valid_test, target_valid_test, test_size=0.5, random_state=12345, stratify = target_valid_test)

Кодируем категориальные признаки

In [16]:
transformer = make_column_transformer(
    (OneHotEncoder(), ['gender', 'geography']),
    remainder='passthrough')
def one_hot(df):
    transformed = transformer.fit_transform(df)
    transformed_df = pd.DataFrame(transformed, columns=transformer.get_feature_names())
    # удаляем по 1 из dummy-признаков
    transformed_df.drop(['onehotencoder__x0_Male', 'onehotencoder__x1_France'], axis = 1, inplace = True)
    return transformed_df

features_train = one_hot(features_train)
features_valid = one_hot(features_valid)
features_test = one_hot(features_test)

Прежде чем обучать модель, масштабируем признаки 

In [17]:
scaler = StandardScaler()
scaler.fit(features_train)

features_train = scaler.transform(features_train)
features_valid = scaler.transform(features_valid) 
features_test = scaler.transform(features_test) 

Обучим модель логистической регрессии и сохраним ее в переменную model_unbalanced

In [18]:
model = LogisticRegression(random_state=12345, C = 2, solver='lbfgs')
model.fit(features_train, target_train)

model_unbalanced_regr = model

Проверим значение f1-score на валидационной выборке. В этом исследовании мы хотим добиться максимального значения f1, поскольку эта метрика отражает одновременно полноту и точность модели

In [19]:
predicted_valid = model.predict(features_valid)
f1_score(target_valid, predicted_valid)

0.3107861060329068

Значение f1 ниже, чем нам бы того хотелось. 

Посчитаем так же значение auc_roc

In [20]:
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]

In [21]:
roc_auc_score(target_valid, probabilities_valid[:, 1])

0.7875760542910633

Значение выше, чем у случайной модели (0.5), но достаточно низкое

Посчитаем вероятность классов с помощью функции predict_proba

Теперь проверим, чему равен f1, для различных значений порога классификации в интервале от 0 до 0.6 с шагом 0.02

In [22]:
for threshold in np.arange(0, 0.6, 0.02):
    predicted_valid = probabilities_one_valid > threshold 
    f1 = f1_score(target_valid, predicted_valid)

    print("Порог = {:.2f} | f1 = {:.3f}".format(
        threshold, f1))

Порог = 0.00 | f1 = 0.339
Порог = 0.02 | f1 = 0.340
Порог = 0.04 | f1 = 0.356
Порог = 0.06 | f1 = 0.379
Порог = 0.08 | f1 = 0.400
Порог = 0.10 | f1 = 0.426
Порог = 0.12 | f1 = 0.445
Порог = 0.14 | f1 = 0.463
Порог = 0.16 | f1 = 0.495
Порог = 0.18 | f1 = 0.502
Порог = 0.20 | f1 = 0.508
Порог = 0.22 | f1 = 0.523
Порог = 0.24 | f1 = 0.524
Порог = 0.26 | f1 = 0.521
Порог = 0.28 | f1 = 0.523
Порог = 0.30 | f1 = 0.505
Порог = 0.32 | f1 = 0.486
Порог = 0.34 | f1 = 0.468
Порог = 0.36 | f1 = 0.446
Порог = 0.38 | f1 = 0.443
Порог = 0.40 | f1 = 0.429
Порог = 0.42 | f1 = 0.402
Порог = 0.44 | f1 = 0.357
Порог = 0.46 | f1 = 0.340
Порог = 0.48 | f1 = 0.328
Порог = 0.50 | f1 = 0.311
Порог = 0.52 | f1 = 0.300
Порог = 0.54 | f1 = 0.293
Порог = 0.56 | f1 = 0.248
Порог = 0.58 | f1 = 0.225


Наибольшее значение f1 получаем при пороге равном 0.24. Значение метрики уже значительно выше, но все еще недостаточно, чтобы считать нашу модель точной.

Теперь проверим какие значения f1 даст нам модель random forrest

In [23]:
model = RandomForestClassifier(random_state=12345)
model.fit(features_train, target_train)

predicted_valid = model.predict(features_valid)
f1_score(target_valid, predicted_valid)

print('f1: ', f1_score(target_valid, predicted_valid))
print('auc-roc: ', roc_auc_score(target_valid, model.predict_proba(features_valid)[:, 1]))

f1:  0.5946745562130178
auc-roc:  0.8603934193023943


Получаем значения f1 и auc_roc выше, чем у линейной регрессии. Теперь поэкспериментируем с гиперпаоаметрами - количеством деревьев и их глубиной

In [24]:
best_model = None
best_est = 0
best_depth = 0
best_result = 0
for est in range(10, 101, 10):
    for depth in range(1,21):
        model = RandomForestClassifier(random_state=12345, n_estimators= est, max_depth = depth)
        model.fit(features_train, target_train)
        predictions_valid = model.predict(features_valid)
        result = f1_score(target_valid, predictions_valid)
        if result > best_result:
            best_result = result
            best_prediction = predictions_valid
            best_est = est
            best_model = model
            best_depth = depth
model_unbalanced_forrest = best_model

print("f1 наилучшей модели на валидационной выборке:", best_result)
print("Кол-во деревьев наилучшей модели на валидационной выборке:", best_est) 
print("Глубина наилучшей модели на валидационной выборке:", best_depth) 

f1 наилучшей модели на валидационной выборке: 0.613095238095238
Кол-во деревьев наилучшей модели на валидационной выборке: 100
Глубина наилучшей модели на валидационной выборке: 16


Для выбранной модели посчитаем так же auc-roc

In [25]:
roc_auc_score(target_valid, best_model.predict_proba(features_valid)[:, 1])

0.8642053712188393

**Вывод**

Для несбалансированной выборки модель случайного леса показывает результаты (f1 и auc) заметно лучше, чем линейная регрессия.

Попробуем улучшить их, устранив дисбаланс классов

## Борьба с дисбалансом

Чтобы устранить дисбаланс, воспользуемся методом upsampling. Ранее мы уже выяснили, что положительные и отрицательные объекты соотносятся приблизительно как 1 к 4

In [26]:
def upsample(features, target, repeat):
#создадим переменные для разных значений классов 
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
#уравняем количество положительных и отрицательных объектов в выборке
    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
#перемешаем выборку    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled

In [27]:

features_train, features_valid_test, target_train, target_valid_test = train_test_split(
    features_unbalanced, target_unbalanced, test_size=0.4, random_state=12345, stratify = target)

features_valid, features_test, target_valid, target_test = train_test_split(
    features_valid_test, target_valid_test, test_size=0.5, random_state=12345, stratify = target_valid_test)

#применим функцию к переменным features и target, затем разделим выборку
features_train, target_train = upsample(features_train, target_train, 4)

Проверим, какой получился баланс для обучающей выборки 

In [28]:
target_train.value_counts(normalize = True)

1    0.50569
0    0.49431
Name: exited, dtype: float64

Далее проделаем те же шаги, что и в исследовании до устранения дисбаланса

In [29]:
transformer = make_column_transformer(
    (OneHotEncoder(), ['gender', 'geography']),
    remainder='passthrough')
def one_hot(df):
    transformed = transformer.fit_transform(df)
    transformed_df = pd.DataFrame(transformed, columns=transformer.get_feature_names())
    # удаляем по 1 из dummy-признаков
    transformed_df.drop(['onehotencoder__x0_Male', 'onehotencoder__x1_France'], axis = 1, inplace = True)
    return transformed_df

features_train = one_hot(features_train)
features_valid = one_hot(features_valid)
features_test = one_hot(features_test)

In [30]:
scaler = StandardScaler()
scaler.fit(features_train)

features_train = scaler.transform(features_train)
features_valid = scaler.transform(features_valid) 
features_test = scaler.transform(features_test) 

Обучим модель логистической регрессии

In [31]:
model = LogisticRegression(random_state=12345, C = 2, solver='lbfgs')
model.fit(features_train, target_train)

predicted_valid = model.predict(features_valid)
print('f1: ', f1_score(target_valid, predicted_valid))
print('auc-roc: ', roc_auc_score(target_valid, model.predict_proba(features_valid)[:,1]))
#обученную модель сохраним в переменную upsampled_model
upsampled_model = model

f1:  0.5246753246753246
auc-roc:  0.7937851019804907


In [32]:
model = GradientBoostingClassifier(random_state=12345)
model.fit(features_train, target_train)

predicted_valid = model.predict(features_valid)
print('f1: ', f1_score(target_valid, predicted_valid))
print('auc-roc: ', roc_auc_score(target_valid, model.predict_proba(features_valid)[:,1]))


f1:  0.6371681415929203
auc-roc:  0.8787996354320624


f1-score стал выше. 

Теперь поисследуем, каким будет f1 для модели случайного леса

In [33]:
model = RandomForestClassifier(random_state=12345)
model.fit(features_train, target_train)

predicted_valid = model.predict(features_valid)
f1_score(target_valid, predicted_valid)


print('f1: ', f1_score(target_valid, predicted_valid))
print('auc-roc: ', roc_auc_score(target_valid, model.predict_proba(features_valid)[:, 1]))

f1:  0.6179921773142113
auc-roc:  0.8589793329392058


f1 и auc-roc изменились незначительно по сравнению с несбалансированной выборкой

На несбалансированной выборке подбор гиперпараметров помог нам улучшить результат.Повторим эту процедуру

In [34]:
best_model = None
best_est = 0
best_depth = 0
best_result = 0
for est in range(50, 101, 5):
    for depth in range(5,20):
        model = RandomForestClassifier(random_state=12345, n_estimators= est, max_depth = depth)
        model.fit(features_train, target_train)
        predictions_valid = model.predict(features_valid)
        result = f1_score(target_valid, predictions_valid)
        if result > best_result:
            best_result = result
            best_prediction = predictions_valid
            best_est = est
            best_model = model
            best_depth = depth

print("f1 наилучшей модели на валидационной выборке:", best_result)
print("Кол-во деревьев наилучшей модели на валидационной выборке:", best_est) 
print("Глубина наилучшей модели на валидационной выборке:", best_depth) 

f1 наилучшей модели на валидационной выборке: 0.6492622020431328
Кол-во деревьев наилучшей модели на валидационной выборке: 75
Глубина наилучшей модели на валидационной выборке: 11


f1 оказался несколько выше, чем с дефолтными параметрами
Сохраним обученную модель в переменную model_upsampled_forrest

In [35]:
model_upsampled_forrest = best_model



Теперь проверим, каков будет результат, если применить технику downsampling

In [36]:
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345)
    return features_downsampled, target_downsampled

In [37]:
features_train, features_valid_test, target_train, target_valid_test = train_test_split(
    features_unbalanced, target_unbalanced, test_size=0.4, random_state=12345, stratify = target)

features_valid, features_test, target_valid, target_test = train_test_split(
    features_valid_test, target_valid_test, test_size=0.5, random_state=12345, stratify = target_valid_test)

#применим функцию к переменным features и target, затем разделим выборку
features_train, target_train = downsample(features_train, target_train, 0.25)

Проверим, какой получился баланс для обучающей выборки 

In [38]:
target_train.value_counts(normalize = True)

1    0.505795
0    0.494205
Name: exited, dtype: float64

Далее проделаем те же шаги, что и ранее

In [39]:
transformer = make_column_transformer(
    (OneHotEncoder(), ['gender', 'geography']),
    remainder='passthrough')
def one_hot(df):
    transformed = transformer.fit_transform(df)
    transformed_df = pd.DataFrame(transformed, columns=transformer.get_feature_names())
    # удаляем по 1 из dummy-признаков
    transformed_df.drop(['onehotencoder__x0_Male', 'onehotencoder__x1_France'], axis = 1, inplace = True)
    return transformed_df

features_train = one_hot(features_train)
features_valid = one_hot(features_valid)
features_test = one_hot(features_test)

In [40]:
scaler = StandardScaler()
scaler.fit(features_train)

features_train = scaler.transform(features_train)
features_valid = scaler.transform(features_valid) 
features_test = scaler.transform(features_test) 

In [41]:
model = LogisticRegression(random_state=12345, solver='lbfgs')
model.fit(features_train, target_train)

predicted_valid = model.predict(features_valid)
print('f1: ', f1_score(target_valid, predicted_valid))
print('auc-roc: ', roc_auc_score(target_valid, model.predict_proba(features_valid)[:, 1]))


f1:  0.5275862068965517
auc-roc:  0.7925796260715341


Результат близок тому, что был получен при методе upsample и ниже того, которого мы хотим добиться

Теперь обучим модель случаного леса

In [42]:
model = RandomForestClassifier(random_state=12345)
model.fit(features_train, target_train)

predicted_valid = model.predict(features_valid)
f1_score(target_valid, predicted_valid)


print('f1: ', f1_score(target_valid, predicted_valid))
print('auc-roc: ', roc_auc_score(target_valid, model.predict_proba(features_valid)[:, 1]))

f1:  0.6083650190114068
auc-roc:  0.8581140999113214


In [43]:
best_model = None
best_est = 0
best_depth = 0
best_result = 0
for est in range(50, 101, 10):
    for depth in range(1,31):
        model = RandomForestClassifier(random_state=12345, n_estimators= est, max_depth = depth)
        model.fit(features_train, target_train)
        predictions_valid = model.predict(features_valid)
        result = f1_score(target_valid, predictions_valid)
        if result > best_result:
            best_result = result
            best_prediction = predictions_valid
            best_est = est
            best_model = model
            best_depth = depth

print("f1 наилучшей модели на валидационной выборке:", best_result)
print("Кол-во деревьев наилучшей модели на валидационной выборке:", best_est) 
print("Глубина наилучшей модели на валидационной выборке:", best_depth) 

f1 наилучшей модели на валидационной выборке: 0.6317829457364341
Кол-во деревьев наилучшей модели на валидационной выборке: 60
Глубина наилучшей модели на валидационной выборке: 9


Метрики качества выше, чем те, что были на несбалансированной выборке. Но немного ниже чем результат для upsampling

### В качестве эксперимента попробуем не писать самостоятельно функцию, а воспользоваться аргументом class_weight при вызове логистической регрессии и случайного леса. В аргумент запишем, что у отрицательных объектов будет вес 1, а у положительных 4

In [44]:
features_train, features_valid_test, target_train, target_valid_test = train_test_split(
    features_unbalanced, target_unbalanced, test_size=0.4, random_state=12345, stratify = target)

features_valid, features_test, target_valid, target_test = train_test_split(
    features_valid_test, target_valid_test, test_size=0.5, random_state=12345, stratify = target_valid_test)

In [45]:
transformer = make_column_transformer(
    (OneHotEncoder(), ['gender', 'geography']),
    remainder='passthrough')
def one_hot(df):
    transformed = transformer.fit_transform(df)
    transformed_df = pd.DataFrame(transformed, columns=transformer.get_feature_names())
    # удаляем по 1 из dummy-признаков
    transformed_df.drop(['onehotencoder__x0_Male', 'onehotencoder__x1_France'], axis = 1, inplace = True)
    return transformed_df

features_train = one_hot(features_train)
features_valid = one_hot(features_valid)
features_test = one_hot(features_test)

In [46]:
scaler = StandardScaler()
scaler.fit(features_train)

features_train = scaler.transform(features_train)
features_valid = scaler.transform(features_valid) 
features_test = scaler.transform(features_test) 

In [47]:
model = LogisticRegression(random_state=12345, solver='lbfgs', class_weight = {0:1, 1:4})
model.fit(features_train, target_train)

predicted_valid = model.predict(features_valid)
print('f1: ', f1_score(target_valid, predicted_valid))
print('auc-roc: ', roc_auc_score(target_valid, model.predict_proba(features_valid)[:,1]))
#обученную модель сохраним в переменную upsampled_model
upsampled_model = model

f1:  0.5246753246753246
auc-roc:  0.7937851019804907


Результат близкий к upsampling и downsampling, но немного ниже

In [48]:
model = RandomForestClassifier(random_state=12345, class_weight = {0:1, 1:4})
model.fit(features_train, target_train)

predicted_valid = model.predict(features_valid)
f1_score(target_valid, predicted_valid)


print('f1: ', f1_score(target_valid, predicted_valid))
print('auc-roc: ', roc_auc_score(target_valid, model.predict_proba(features_valid)[:, 1]))

f1:  0.5749235474006116
auc-roc:  0.8615211474036851


Также как и для логистической регрессии результат близок, но несколько ниже чем для upsampling и downsampling, которые мы произвели вручную 

###  Вывод


  После того, как мы устранили дисбаланс классов, значение метрики f1 значительно увеличилось как для логистической регресси, так и для случайного леса. А значит количество ложно отрицательных и ложно положительных предсказаний стало меньше. При этом метод увеличения выборки дал результаты лучше, чем метод уменьшения, а модель случайного леса показала себя лучше чем логистическая регрессия

Выбираем модель случайного леса, обученную на увеличенной выборке. Ее сохранил в переменную model_upsampled_forrest

## Тестирование модели

Проверим на тестовой выборке значение f1 для модели upsampled_model с порогом классификации равном 0.46

In [49]:
predicted_test = model_upsampled_forrest.predict(features_test)

f1_score(target_test, predicted_test)

0.5850052798310453

Рассчитаем также значение auc roc

In [50]:
roc_auc_score(target_test, model.predict_proba(features_test)[:, 1])

0.8425621306977239

Результаты оказались заметно ниже, чем на валидационной выборке, но выше той цели, которую мы себе ставили.  

Мы добились приемлемого показателя метрики f1. Можем ожидать, что с помощью модели бета-банк сохранит многих клиентов и сэкономит деньги на привлечение новых